In [12]:
import torch.nn as nn
from torchinfo import summary

# Required code for ResNet

In [11]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv1d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm1d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv1d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm1d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# Implementing FirstLayersBlock

In [70]:
from prixfixe.prixfixe import FirstLayersBlock

class WrongResnetFirstLayersBlock(FirstLayersBlock):
    def __init__(self, 
                 in_channels: int, 
                 out_channels: int, 
                 seqsize: int):
        super().__init__(in_channels=in_channels,
                         out_channels=out_channels,
                         seqsize=seqsize)
        kernel_size = 7
        stride = 1
        padding = "same"

        self.conv1 = nn.Sequential(
                        nn.Conv1d(in_channels,
                                  out_channels,
                                  kernel_size = kernel_size,
                                  stride = stride, 
                                  padding = padding),
                        nn.BatchNorm1d(64),
                        nn.ReLU())

##### In this scenario, we failed to implement several methods required for FirstLayersBlock. Abstract class won't allow us to use such a module

In [71]:
WrongResnetFirstLayersBlock(in_channels=4,
                       out_channels=64, 
                       seqsize=110)

TypeError: Can't instantiate abstract class WrongResnetFirstLayersBlock with abstract method forward

In [74]:
from prixfixe.prixfixe import FirstLayersBlock

class ResnetFirstLayersBlock(FirstLayersBlock):
    def __init__(self, 
                 in_channels: int, 
                 out_channels: int, 
                 seqsize: int):
        super().__init__(in_channels=in_channels,
                         out_channels=out_channels,
                         seqsize=seqsize)
        kernel_size = 7
        stride = 1
        padding = "same"

        self.conv1 = nn.Sequential(
                        nn.Conv1d(in_channels,
                                  out_channels,
                                  kernel_size = kernel_size,
                                  stride = stride, 
                                  padding = padding),
                        nn.BatchNorm1d(out_channels),
                        nn.ReLU())
    def forward(self, x):
        x = self.conv1(x)
        return x

In [76]:
first = ResnetFirstLayersBlock(in_channels=4,
                       out_channels=256, 
                       seqsize=110)
summary(first, (1, 4, 110))

Layer (type:depth-idx)                   Output Shape              Param #
ResnetFirstLayersBlock                   [1, 256, 110]             --
├─Sequential: 1-1                        [1, 256, 110]             --
│    └─Conv1d: 2-1                       [1, 256, 110]             7,424
│    └─BatchNorm1d: 2-2                  [1, 256, 110]             512
│    └─ReLU: 2-3                         [1, 256, 110]             --
Total params: 7,936
Trainable params: 7,936
Non-trainable params: 0
Total mult-adds (M): 0.82
Input size (MB): 0.00
Forward/backward pass size (MB): 0.45
Params size (MB): 0.03
Estimated Total Size (MB): 0.48

# Implementing CoreBlock

In [77]:
from prixfixe.prixfixe import CoreBlock

class ResNetCoreBlock(CoreBlock):
    def __init__(self, 
                 in_channels: int, 
                 out_channels: int, 
                 seqsize: int):
        super().__init__(in_channels=in_channels,
              out_channels=out_channels,
              seqsize=seqsize)
        block =  ResidualBlock
        layers =  [3,4,6,3]
        self.inplanes = 64
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, out_channels, layers[3], stride = 2)
    
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm1d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [82]:
core = ResNetCoreBlock(in_channels=64, 
               out_channels=512,
               seqsize=110)

summary(core, (1, 64, 110))

Layer (type:depth-idx)                   Output Shape              Param #
ResNetCoreBlock                          [1, 512, 14]              --
├─Sequential: 1-1                        [1, 64, 110]              --
│    └─ResidualBlock: 2-1                [1, 64, 110]              --
│    │    └─Sequential: 3-1              [1, 64, 110]              12,480
│    │    └─Sequential: 3-2              [1, 64, 110]              12,480
│    │    └─ReLU: 3-3                    [1, 64, 110]              --
│    └─ResidualBlock: 2-2                [1, 64, 110]              --
│    │    └─Sequential: 3-4              [1, 64, 110]              12,480
│    │    └─Sequential: 3-5              [1, 64, 110]              12,480
│    │    └─ReLU: 3-6                    [1, 64, 110]              --
│    └─ResidualBlock: 2-3                [1, 64, 110]              --
│    │    └─Sequential: 3-7              [1, 64, 110]              12,480
│    │    └─Sequential: 3-8              [1, 64, 110]            

# Implementing FinalLayersBlock

In [83]:
from prixfixe.prixfixe import FinalLayersBlock

class WrongResNetFinalLayersBlock(FinalLayersBlock):
    def __init__(self, 
                 in_channels: int, 
                 seqsize: int):
        super().__init__(in_channels=in_channels,
                         seqsize=seqsize)
        
        self.avgpool = nn.AvgPool1d(1) # global average pooling
        self.linear = nn.Sequential(
              nn.Linear(in_channels, in_channels),
              nn.ReLU(),
              nn.Linear(in_channels, 1)
            )

    def forward(self, x):
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

In [85]:
WrongResNetFinalLayersBlock(in_channels=512, seqsize=110)

TypeError: Can't instantiate abstract class WrongResNetFinalLayersBlock with abstract method train_step

##### _FinalLayersBlock_ must implement loss calculation logic, so we have to define train_step (for other blocks the default implementation was used)

In [86]:
import torch
from typing import Any 

from prixfixe.prixfixe import FinalLayersBlock

class ResNetFinalLayersBlock(FinalLayersBlock):
    def __init__(self, 
                 in_channels: int, 
                 seqsize: int):
        super().__init__(in_channels=in_channels,
                         seqsize=seqsize)
        
        self.avgpool = nn.AvgPool1d(1) # global average pooling
        self.linear = nn.Sequential(
              nn.Linear(in_channels * self.seqsize, in_channels),
              nn.ReLU(),
              nn.Linear(in_channels, 1)
            )
        self.criterion = torch.nn.MSELoss()

    def forward(self, x):
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x
    
    def train_step(self, batch: dict[str, Any]) -> tuple[torch.Tensor, torch.Tensor]:
        x = batch["x"].to(self.device)
        y_pred = self.forward(x)
        y = batch["y"].to(self.device)
        loss = self.criterion(y, y_pred)
        
        return y, loss

In [87]:
final = ResNetFinalLayersBlock(in_channels=512, seqsize=110)
summary(final, (1, 512, 110))

Layer (type:depth-idx)                   Output Shape              Param #
ResNetFinalLayersBlock                   [1, 1]                    --
├─AvgPool1d: 1-1                         [1, 512, 110]             --
├─Sequential: 1-2                        [1, 1]                    --
│    └─Linear: 2-1                       [1, 512]                  28,836,352
│    └─ReLU: 2-2                         [1, 512]                  --
│    └─Linear: 2-3                       [1, 1]                    513
Total params: 28,836,865
Trainable params: 28,836,865
Non-trainable params: 0
Total mult-adds (M): 28.84
Input size (MB): 0.23
Forward/backward pass size (MB): 0.00
Params size (MB): 115.35
Estimated Total Size (MB): 115.58

# Joining blocks in PrixFixe

In [88]:
from prixfixe.prixfixe import PrixFixeNet

##### Pay attention to the fact that _FirstLayersBlock_ and _CoreBlock_ have the ".infer_outseqsize()” method to allow passing this information to the next blocks in the pipeline. This method is already implemented in abstract class so there is no need for teams to rewrite it

In [90]:
first = ResnetFirstLayersBlock(in_channels=4,
                               out_channels=64,
                               seqsize=150)
core = ResNetCoreBlock(in_channels=first.out_channels,
                       out_channels=512,
                       seqsize=first.infer_outseqsize())
final = ResNetFinalLayersBlock(in_channels=core.out_channels,
                               seqsize=core.infer_outseqsize())
resnet = PrixFixeNet(first=first,
            core=core,
            final=final,
            generator=torch.Generator())

##### We can check the correctness of the final model

In [91]:
resnet.check()

Checking forward pass
Forward is OK
Checking training step
Training step is OK


In [92]:
summary(resnet, (1, 4, 150))

Layer (type:depth-idx)                        Output Shape              Param #
PrixFixeNet                                   [1, 1]                    --
├─ResnetFirstLayersBlock: 1-1                 --                        --
│    └─Sequential: 2-1                        [1, 64, 150]              --
│    │    └─Conv1d: 3-1                       [1, 64, 150]              1,856
│    │    └─BatchNorm1d: 3-2                  [1, 64, 150]              128
│    │    └─ReLU: 3-3                         [1, 64, 150]              --
├─ResNetCoreBlock: 1-2                        --                        --
│    └─Sequential: 2-2                        [1, 64, 150]              --
│    │    └─ResidualBlock: 3-4                [1, 64, 150]              24,960
│    │    └─ResidualBlock: 3-5                [1, 64, 150]              24,960
│    │    └─ResidualBlock: 3-6                [1, 64, 150]              24,960
│    └─Sequential: 2-3                        [1, 128, 75]              --
│   

##### And check if it is compatible with blocks from other teams.

In [96]:
from prixfixe.autosome import AutosomeCoreBlock

first = ResnetFirstLayersBlock(in_channels=4,
                               out_channels=256,
                               seqsize=150)
core =  AutosomeCoreBlock(in_channels=first.out_channels,
                       out_channels=64,
                       seqsize=first.infer_outseqsize())
final = ResNetFinalLayersBlock(in_channels=core.out_channels,
                               seqsize=core.infer_outseqsize())
resnet_with_autosome_core_block = PrixFixeNet(first=first,
            core=core,
            final=final,
            generator=torch.Generator())

In [97]:
resnet_with_autosome_core_block.check()

Checking forward pass
Forward is OK
Checking training step
Training step is OK


In [98]:
summary(resnet_with_autosome_core_block, (1, 4, 150))

Layer (type:depth-idx)                        Output Shape              Param #
PrixFixeNet                                   [1, 1]                    --
├─ResnetFirstLayersBlock: 1-1                 --                        --
│    └─Sequential: 2-1                        [1, 256, 150]             --
│    │    └─Conv1d: 3-1                       [1, 256, 150]             7,424
│    │    └─BatchNorm1d: 3-2                  [1, 256, 150]             512
│    │    └─ReLU: 3-3                         [1, 256, 150]             --
├─AutosomeCoreBlock: 1-2                      --                        --
│    └─ModuleDict: 2-2                        --                        --
│    │    └─Sequential: 3-4                   [1, 256, 150]             337,984
│    │    └─Sequential: 3-5                   [1, 128, 150]             459,008
│    │    └─Sequential: 3-6                   [1, 128, 150]             173,856
│    │    └─Sequential: 3-7                   [1, 128, 150]             229,

In [99]:
from prixfixe.bhi import BHICoreBlock

first = ResnetFirstLayersBlock(in_channels=4,
                               out_channels=256,
                               seqsize=150)
core =  BHICoreBlock(
    in_channels = first.out_channels,
    out_channels = 320,
    seqsize = first.infer_outseqsize(),
    lstm_hidden_channels = 320,
    kernel_sizes = [9, 15],
    pool_size = 1,
    dropout1 = 0.2,
    dropout2 = 0.5
    )
final = ResNetFinalLayersBlock(in_channels=core.out_channels,
                               seqsize=core.infer_outseqsize())
resnet_with_bhi_core_block = PrixFixeNet(first=first,
            core=core,
            final=final,
            generator=torch.Generator())

In [100]:
resnet_with_bhi_core_block.check()

Checking forward pass
Forward is OK
Checking training step
Training step is OK


In [101]:
summary(resnet_with_bhi_core_block, (1, 4, 150))

Layer (type:depth-idx)                   Output Shape              Param #
PrixFixeNet                              [1, 1]                    --
├─ResnetFirstLayersBlock: 1-1            --                        --
│    └─Sequential: 2-1                   [1, 256, 150]             --
│    │    └─Conv1d: 3-1                  [1, 256, 150]             7,424
│    │    └─BatchNorm1d: 3-2             [1, 256, 150]             512
│    │    └─ReLU: 3-3                    [1, 256, 150]             --
├─BHICoreBlock: 1-2                      --                        --
│    └─LSTM: 2-2                         [1, 150, 640]             1,479,680
│    └─ModuleList: 2-3                   --                        --
│    │    └─ConvBlock: 3-4               [1, 160, 150]             921,760
│    │    └─ConvBlock: 3-5               [1, 160, 150]             1,536,160
│    └─Dropout: 2-4                      [1, 320, 150]             --
├─ResNetFinalLayersBlock: 1-3            --                   